# IMU Feature Extraction Pipeline

## Setup

In [ ]:
from pathlib import Path
import json
import re

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.utils.time import timedelta_to_time
from biopsykit.signals.imu.feature_extraction.static_moments import compute_features

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from joblib import Parallel
from tqdm.notebook import tqdm

from carwatch_analysis.datasets import CarWatchDatasetRaw
from carwatch_analysis.data_processing.imu import process_night
from carwatch_analysis.exceptions import (
    ImuDataNotFoundException,
    NoSuitableImuDataFoundException,
    DateNotAvailableException,
)

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
base_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])

base_path

In [ ]:
dataset = CarWatchDatasetRaw(base_path)
dataset

## IMU Data Processing

### Processing parameters

In [ ]:
compute_imu_endpoints = True
compute_sm_features = False

thres = 100
window_sec = 5
overlap_percent = 0.9

epoch_length = 30

In [ ]:
for subset in tqdm(dataset.groupby(["subject", "night"])):
    if not subset.subject_folder_path.exists():
        continue
    export_path = subset.subject_folder_path.joinpath("processed")
    export_path.mkdir(exist_ok=True)
    subject_id = subset.index["subject"][0]
    night_id = subset.index["night"][0]

    feature_file = export_path.joinpath(f"imu_static_moment_features_{subject_id}_{night_id}.csv")
    endpoint_file = export_path.joinpath(f"sleep_endpoints_{subject_id}_{night_id}.csv")

    endpoints_imu, sm_features = process_night(
        subset,
        compute_imu_endpoints=compute_imu_endpoints,
        compute_sm_features=compute_sm_features,
        thres=thres,
        window_sec=window_sec,
        overlap_percent=overlap_percent,
        epoch_length=epoch_length,
        compare_endpoints=True,
        export_path=export_path,
    )

    if endpoints_imu is not None:
        endpoints_imu.to_csv(endpoint_file)
    if sm_features is not None:
        sm_features.to_csv(feature_file)